Все импорты должны выполняться, особенно если вы запускаете это внутри докера

In [4]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Populating the interactive namespace from numpy and matplotlib


In [5]:
import pandas as pd
import pandas.io.sql as psql

Вспомогательные функции для запроса данных из базы

In [6]:
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")

def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

def get_uid():
    return run_sql("SELECT uid FROM users WHERE data_loaded")

In [4]:
# Всего у нас вот столько таблиц

run_sql("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")

,table_name
0,groups
1,likes
2,posts
3,users_friends
4,users_groups
5,users


In [8]:
run_sql("Select * FROM users where data_loaded limit 5")

,first_name,last_name,sex,nickname,domain,bdate,city,country,photo_50,photo_100,photo_200_orig,can_write_private_message,status,data_loaded,mobile_phone,home_phone,group_list_loaded,comments_loaded,is_leader
uid,,,,,,,,,,,,,,,,,,,
94377664,Валерия ...,Салимовская,1,,lera_salimovskaya,,280,2,https://sun9-1.userapi.com/c626929/v626929664/...,https://pp.userapi.com/c626929/v626929664/3a37...,https://pp.userapi.com/c626929/v626929664/3a37...,0,,True,,,True,True,0
52859655,Анастасия ...,Шлыкова,1,~Nasty~,n_sh_96,22.11,280,2,https://pp.userapi.com/c855636/v855636861/3e59...,https://pp.userapi.com/c855636/v855636861/3e58...,https://pp.userapi.com/c855636/v855636861/3e58...,1,🙈,True,,,True,True,0
96782258,Екатерина ...,Долгополая,1,,id96782258,,280,2,https://pp.userapi.com/c837324/v837324258/1fb3...,https://sun9-1.userapi.com/c837324/v837324258/...,https://pp.userapi.com/c837324/v837324258/1faf...,1,Жизнь прекрасна),True,,,True,True,0
136709675,Veronika ...,Shíbalová,1,,id136709675,12.5.1996,0,215,https://pp.userapi.com/c4334/u136709675/e_7f2b...,https://pp.userapi.com/c4334/u136709675/d_b605...,https://pp.userapi.com/c4334/u136709675/a_335f...,1,,True,,,True,True,0
96895011,Инна ...,Свирская,1,,id96895011,25.9,280,2,https://pp.userapi.com/c628418/v628418011/dc69...,https://pp.userapi.com/c628418/v628418011/dc68...,https://pp.userapi.com/c628418/v628418011/dc65...,1,"салон красоты Для Вас , Харьков, пр. Гагарина...",True,+380676260409,+380577509552,True,True,0
106360483,Tomas ...,Georges,2,,tomasgeorges,22.9,280,2,https://pp.userapi.com/c626817/v626817483/365f...,https://pp.userapi.com/c626817/v626817483/365f...,https://pp.userapi.com/c626817/v626817483/365e...,0,"С одной, и на всю жизнь. Вот к чему нужно стре...",True,,,True,True,0
116329452,Олег ...,Яковенко,2,,7yakovenko7,14.10,2722607,79,https://pp.userapi.com/c626226/v626226452/31c4...,https://pp.userapi.com/c626226/v626226452/31c4...,https://pp.userapi.com/c626226/v626226452/31c4...,1,И мы с братом устроим свой Баден-Баден!,True,,,True,True,0
162556252,Александра ...,Шалупова,1,,id162556252,26.1,185,1,https://pp.userapi.com/c849420/v849420867/1191...,https://pp.userapi.com/c849420/v849420867/1191...,https://pp.userapi.com/c849420/v849420867/1191...,0,Пусть эта зима будет самой волшебной. Пусть сб...,True,,,True,True,0
163483844,Илья ...,Шиков,2,,id163483844,22.9.2003,1397,2,https://pp.userapi.com/c846021/v846021389/78b4...,https://pp.userapi.com/c846021/v846021389/78b4...,https://pp.userapi.com/c846021/v846021389/78b4...,1,,True,,,True,True,0


In [9]:
users = run_sql("""Select uid, first_name, last_name, sex, is_leader 
                FROM users 
                where data_loaded and group_list_loaded and comments_loaded""")

users.to_csv('../data/raw_data/users.csv', index=False)

In [14]:
run_sql("""Select * 
           FROM groups 
           limit 5""")

,gid,name,screen_name,is_closed,type,photo,photo_medium,photo_big
0,48457106,ТМ ВиТиеВаТо / Украшения из проволоки,tm_vitievato,0,page,https://pp.userapi.com/c633231/v633231909/17c3...,https://pp.userapi.com/c633231/v633231909/17c3...,https://pp.userapi.com/c633231/v633231909/17c3...
1,52159629,Реальные Пацаны (Новые Серии) | Универ (Новый),tnt_realnue,0,page,https://pp.userapi.com/mv-k7Kdy6ulFCEW6sO2bmuN...,https://pp.userapi.com/cZtc5wEUIXozLniDSp8Uf9o...,https://pp.userapi.com/TffOP3rm5582WEUn7tnedwx...
2,52914122,Ростов-город ● 161 ● Ростов-Дон,rostov_gorod_161_rostov_don,0,page,https://pp.userapi.com/c841329/v841329164/3091...,https://pp.userapi.com/c841329/v841329164/3091...,https://pp.userapi.com/c841329/v841329164/3091...
3,53376961,HOT WHEELS Exclusive Сar Shop,hwexclusive,0,page,https://pp.userapi.com/5OLTm6mWr9CJVOGfg53tQfN...,https://pp.userapi.com/rEhCO9T1y4vZfrC4JTEgwW_...,https://pp.userapi.com/Jwqeyx2QPcfquCtLLy7sHBB...
4,53995491,Booking Machine Shop,bookingmachineshop,0,page,https://sun1-86.userapi.com/c837139/v837139778...,https://sun1-14.userapi.com/c837139/v837139778...,https://sun1-21.userapi.com/c837139/v837139778...
5,54460659,FIZRUK Прокат Велосипедов Казань,kazanfizruk,0,page,https://pp.userapi.com/c848636/v848636707/1851...,https://pp.userapi.com/c848636/v848636707/1851...,https://pp.userapi.com/c848636/v848636707/1851...
6,56096709,Моrе FEST 2017 (FRESH FESТ),ffpage,0,page,https://pp.userapi.com/c638931/v638931065/3176...,https://pp.userapi.com/c638931/v638931065/3176...,https://pp.userapi.com/c638931/v638931065/3176...
7,56862106,Свадебный фотограф в Москве | Наталья Байрамова,nnnbayramova,0,page,https://pp.userapi.com/c845124/v845124691/4980...,https://pp.userapi.com/c845124/v845124691/4980...,https://pp.userapi.com/c845124/v845124691/4980...
8,60044760,Ёлочные Игрушки к Новому Году,club60044760,0,page,https://pp.userapi.com/sNficLrHq-pJmR-EzAGTnE6...,https://pp.userapi.com/-n-ySMpHeIuYQAIAMaIttuF...,https://pp.userapi.com/3L0edKPJ5Qa8g_5b3gcqlgo...
9,62077973,"Cat s eye shop | Неформальный стиль, украшения|",catseye.gothshop,0,page,https://pp.userapi.com/c639119/v639119695/6115...,https://pp.userapi.com/c639119/v639119695/6115...,https://pp.userapi.com/c639119/v639119695/6114...


In [17]:
df = run_sql("""Select gid, name, screen_name 
                FROM groups 
                where is_closed = 0""")

df.to_csv('../data/raw_data/groups.csv', index=False)

In [20]:
run_sql("""Select * 
           FROM likes 
           limit 5""")

,id,owner_id,user_id
0,3694,63350736,179907918
1,3958,101192512,335342017
2,205,8520297,4511856
3,3840,101192512,10453460
4,2142,97043772,439713007


In [21]:
df = run_sql("""Select owner_id, user_id 
                FROM likes""")

df = df.groupby('owner_id', as_index=False).agg('count')
df.to_csv('../data/raw_data/likes.csv', index=False)

In [56]:
run_sql("""Select * 
           FROM users_friends 
           limit 5""")

,uid1,uid2
0,5182336,60993
1,60993,5182336
2,5182336,65843
3,65843,5182336
4,5182336,70237


In [4]:
df = run_sql("""Select uid1, count(uid2) 
                FROM users_friends
                group by uid1""")


In [6]:
df.to_csv('../data/raw_data/friends.csv', index=False)

In [7]:
run_sql("""Select * 
           FROM users_groups 
           limit 5""")

,gid
uid,
4080406,131631279
4080406,149349881
4080406,35193970
4080406,27121021
4080406,10280580


In [13]:
df = run_sql("""Select * 
                FROM users_groups 
             """)

In [18]:
df.to_csv('../data/raw_data/users_groups.csv', index=False)

In [4]:
run_sql("""Select * 
           FROM posts
           where text <> ''
           limit 5
        """)

,id,from_id,owner_id,date,post_type,text,post_source,comments,is_favorite,likes,reposts,copy_history,attachments,likes_loaded
0,622,2621162,2621162,1367321413,post,Узнаю себя:)),{},{'count': 0},False,{'count': 6},{'count': 2},"[{'id': 55565, 'date': 1367315915, 'text': '',...",{},True
1,1952,3250511,3250511,1274289175,post,sometimes...........I HATE MYSELF!!!!!!!!!!!!!...,{},{'count': 0},False,{'count': 0},{'count': 0},{},{},True
2,1950,3250511,3250511,1274095040,post,разбираем и владеем!,{},{'count': 0},False,{'count': 0},{'count': 0},{},{},True
3,5872,4739874,4739874,1410104966,post,На главной площади #шанхай http://instagram.co...,{},{'count': 0},False,{'count': 11},{'count': 0},{},"[{'type': 'photo', 'photo': {'id': 338824411, ...",True
4,654,3250511,3250511,1235079844,post,Горе от ума.,{},{'count': 0},False,{'count': 0},{'count': 0},{},{},True


In [ ]:
from tqdm import tqdm

total = 8982138
def dict2int(x):
    return x['count']

posts = []

for i in tqdm(range((total//1000000) + 1)):

    df = run_sql("""Select from_id, owner_id, text, comments, likes, reposts
                    FROM posts
                    where text <> '' and likes_loaded
                    limit 1000000 offset {}
                 """.format(i*1000000))

    df['comments'] = df['comments'].apply(dict2int).astype(np.int32)
    df['likes'] = df['likes'].apply(dict2int).astype(np.int32)
    df['reposts'] = df['reposts'].apply(dict2int).astype(np.int32)
    posts.append(df)

df = pd.concat(posts).drop_duplicates()
df.to_csv('../data/raw_data/posts.csv', index=False)

100%|██████████| 9/9 [21:47<00:00, 147.38s/it]


In [ ]:
df = pd.concat(posts)
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [8]:
1

1

In [29]:
def dict2int(x):
    return x['count']

df['comments'] = df['comments'].apply(dict2int).astype(np.int32)
df['likes'] = df['likes'].apply(dict2int).astype(np.int32)
df['reposts'] = df['reposts'].apply(dict2int).astype(np.int32)



In [38]:
df.shape

(1000000, 6)

In [39]:
df.to_csv('../data/raw_data/posts_1kk.csv', index=False)

In [ ]:
1

In [80]:
# from psycopg2 import sql

# conn = get_conn()
# cur = conn.cursor()

# cur.execute(
#     sql.SQL("ALTER TABLE {} DROP COLUMN is_leader")
#         .format(sql.Identifier('users')))
# conn.commit()



In [81]:
# # Добавляем новую колонку напрямую в базу


# from psycopg2 import sql

# conn = get_conn()
# cur = conn.cursor()

# cur.execute(
#     sql.SQL("ALTER TABLE {} ADD COLUMN is_leader INT")
#         .format(sql.Identifier('users')))
# conn.commit()


In [11]:
# Допустим теперь для каждого юзера у нас появились предсказания, запишем их в сабмишн файл 

ids = get_uid()
leaders = np.random.choice(list(ids.index), size=100)


submission = pd.DataFrame({'is_leader':np.zeros(ids.shape[0])}, index=ids.index)
submission.loc[leaders] = 1

submission.to_csv('../data/submission/random_submission.csv')

In [90]:
# # Обновляем колонку в базе 
# # Так делать не нужно, код для Андрея
# from tqdm import tqdm

# conn = get_conn()
# cur = conn.cursor()

# cur.execute(
#         sql.SQL("""UPDATE {} SET is_leader = 0""")
#     .format(sql.Identifier('users'))
# )

# leaders = submission[submission['is_leader'] == 1]
# for uid, is_leader in tqdm(zip(leaders.index, leaders['is_leader'])):
#     cur.execute(
#         sql.SQL("""UPDATE {}
#                 SET is_leader = %s
#                 WHERE
#                    uid = %s;
#                 """)
#             .format(sql.Identifier('users')),
#         [int(is_leader), uid])
# conn.commit()
